# Importing table data from URLs in to Pandas

Like the **read_csv** and **read_excel** methods, Pandas also has a method for reading tables from websites.

In [41]:
from IPython.display import Image
import pandas as pd
import html5lib

***
For the following example, we will try and read a table for rainfall in Oklahoma from [Mesonet](http://www.mesonet.org/index.php/weather/rainfall_table).

![image](../Images/Mesonet.png)

***
The **[read_html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_html.html "pd.read_html")** method accepts the site url and returns a list of DataFrames.

In [29]:
DfFromURL = pd.read_html('https://www.mesonet.org/index.php/weather/rainfall_table')
DfFromURL

[    Station Name  7 Day  10 Day  14 Day  30 Day 60 Day 90 Day  September  \
 0           Acme   3.94    3.98    7.21    7.31   9.73  12.80       5.66   
 1            Ada   0.99    0.99    7.99    7.99  11.25  17.57       6.50   
 2          Altus   2.11    2.35    4.34    4.51   5.31   7.04       2.46   
 3           Alva   1.40    1.40    1.40    1.80  10.52  14.03       1.40   
 4        Antlers   1.01    1.10    7.08    7.79  12.87  15.06       4.77   
 ..           ...    ...     ...     ...     ...    ...    ...        ...   
 115    Wilburton   2.81    3.07    7.93    8.77  12.44  14.74       7.19   
 116       Wister   2.24    2.40   10.26   12.20  17.55  20.46       8.22   
 117     Woodward   1.11    1.11    1.99    2.02   4.58   8.13       1.84   
 118       Wynona   4.36    4.36    4.72    6.39  10.53  17.23       4.46   
 119        Yukon   1.87    1.90    7.21    7.22  15.14  19.51       1.91   
 
     Year to date   2019  
 0          33.05  36.36  
 1          41.87  4

***
Since the returned object is a list, the DataFrame can be accessed by calling the specific element in the list *(0 in the case for a single element)*. Now we have the capability of importing data from a website directly into Pandas!

In [43]:
DfFromURL[0]

,Station Name,7 Day,10 Day,14 Day,30 Day,60 Day,90 Day,September,Year to date,2019
0,Acme,3.94,3.98,7.21,7.31,9.73,12.80,5.66,33.05,36.36
1,Ada,0.99,0.99,7.99,7.99,11.25,17.57,6.50,41.87,40.20
2,Altus,2.11,2.35,4.34,4.51,5.31,7.04,2.46,22.06,26.72
3,Alva,1.40,1.40,1.40,1.80,10.52,14.03,1.40,24.03,34.49
4,Antlers,1.01,1.10,7.08,7.79,12.87,15.06,4.77,42.60,57.01
...,...,...,...,...,...,...,...,...,...,...
115,Wilburton,2.81,3.07,7.93,8.77,12.44,14.74,7.19,48.18,52.90
116,Wister,2.24,2.40,10.26,12.20,17.55,20.46,8.22,54.58,60.84
117,Woodward,1.11,1.11,1.99,2.02,4.58,8.13,1.84,16.78,30.84
118,Wynona,4.36,4.36,4.72,6.39,10.53,17.23,4.46,35.26,*


* * *
However, some issues may arise when trying to read tables from dynamic web pages such as this one from [weather.gov](https://www.weather.gov/sjt/TexasObservationsTable)

![THW](../Images/TexasHourlyWeather.png)

In [47]:
DfFromURL2 = pd.read_html('http://www.weather.gov/sjt/TexasObservationsTable')
print("Number of objects in returned list:",len(DfFromURL2))
DfFromURL2

Number of objects in returned list: 1


[                              0                                     1  \
 0                           NaN                                   NaN   
 1  Current Weather  Hazards Map  Minute By Minute  San Angelo Weather   
 
                        2                              3  
 0                    NaN                            NaN  
 1  Latest Storm  Reports  Regional Temp &  Precip Table  ]

***
Obviously, this is not the table we wanted. This is an instance where the web page is not being rendered prior to pulling the data. 
<br> This problem can be resolved, by using a tool like [selenium](https:www.selenium.dev) to pull the data after the page is fully rendered. 

In [61]:
from selenium import webdriver

driver = webdriver.Chrome('../chromedriver.exe')   #
driver.get("http://www.weather.gov/sjt/TexasObservationsTable")

html = driver.page_source

tables = pd.read_html(html)
data = tables

driver.close()

In [62]:
print("Number of objects in returned list:",len(data))

Number of objects in returned list: 2


***
We can see now that the list of DataFrames objects returned from the **read_html** method has increased to include the table we originally wanted and can now access through the first element in the list.

In [70]:
df = tables[0].copy()
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,Station Name,Weather,Temp,Feels,Dwpt,RH,Visibility,Wind,24Hr Pc,24Hr Lo,24Hr Hi
1,Abilene,Clear,72°F,72°F,67°F,84%,10.0Miles,E 7MPH,0.00In.,61°F,85°F
2,Alpine,Clear,69°F,69°F,55°F,61%,10.0Miles,Calm,0.00In.,55°F,86°F
3,Amarillo,Clear,59°F,59°F,47°F,64%,10.0Miles,ESE 7MPH,0.00In.,52°F,83°F
4,Austin,Few Clouds,75°F,75°F,70°F,84%,10.0Miles,NNW 3MPH,0.00In.,67°F,92°F


***
A little cleanup may be necessary. In this case, the first row will need to be set as the column headers with a subsequent step to drop the first row afterwards. 

In [71]:
df.columns = df.iloc[0]   #Set Column headers to values in first row
df = df[1:]    #Assign DataFrame omitting the the first row
df.head()

,Station Name,Weather,Temp,Feels,Dwpt,RH,Visibility,Wind,24Hr Pc,24Hr Lo,24Hr Hi
1,Abilene,Clear,72°F,72°F,67°F,84%,10.0Miles,E 7MPH,0.00In.,61°F,85°F
2,Alpine,Clear,69°F,69°F,55°F,61%,10.0Miles,Calm,0.00In.,55°F,86°F
3,Amarillo,Clear,59°F,59°F,47°F,64%,10.0Miles,ESE 7MPH,0.00In.,52°F,83°F
4,Austin,Few Clouds,75°F,75°F,70°F,84%,10.0Miles,NNW 3MPH,0.00In.,67°F,92°F
5,Beaumont,Clear,78°F,78°F,75°F,91%,10.0Miles,NE 8MPH,0.00In.,75°F,95°F
